Boston dataset is one of the datasets available in sklearn.
You are given a Training dataset csv file with X train and Y train data. As studied in lecture, your task is to come up with Gradient Descent algorithm and thus predictions for the test dataset given.
Your task is to:
    
1. Code Gradient Descent for N features and come with predictions.
2. Try and test with various combinations of learning rates and number of iterations.
3. Try using Feature Scaling, and see if it helps you in getting better results. 

Read Instructions carefully -

1. Use Gradient Descent as a training algorithm and submit results predicted.
2. Files are in csv format, you can use genfromtxt function in numpy to load data from csv file. Similarly you can use savetxt function to save data into a file.
3. Submit a csv file with only predictions for X test data. File name should not have spaces. File should not have any headers and should only have one column i.e. predictions. Also predictions shouldn't be in exponential form. 
4. Your score is based on coefficient of determination.

# Import data

In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [68]:
train_data = np.loadtxt(r'C:\Users\DELL\Desktop\CodingNinjas\Gradient Descent TASK-2\training_boston_x_y_train.csv',delimiter=',')

In [69]:
train_data.shape

(379, 14)

In [70]:
test_data = np.loadtxt(r'C:\Users\DELL\Desktop\CodingNinjas\Gradient Descent TASK-2\test_boston_xTest.csv',delimiter=',')

In [71]:
test_data.shape

(127, 13)

# Split into input and output values

In [72]:
x = train_data[:, 0:13]
y = train_data[:, 13]

In [73]:
print(x.shape)
print(y.shape)

(379, 13)
(379,)


# Multivariate feature Inclusion

In [74]:
#for training data
#converting data into degree 2 
columns = x.shape[1]
for i in range(columns):
    for j in range(i+1, columns):
        new_col = (x[:, i] * x[:, j]).reshape(-1, 1)
        x = np.append(x, new_col, axis = 1)

for i in range(columns):
    new_col = (x[:, i] * x[:, i]).reshape(-1,1)
    x = np.append(x, new_col, axis=1)

#for test data
#converting data into degree 2 
columns = test_data.shape[1]
for i in range(columns):
    for j in range(i+1,columns):
        new_col = (test_data[:, i] * test_data[:, j]).reshape(-1, 1)
        test_data = np.append(test_data, new_col, axis = 1)
        
for i in range(columns):
    new_col = (test_data[:, i] * test_data[:, i]).reshape(-1,1)
    test_data = np.append(test_data, new_col, axis=1)

# Add a new column of ones 

In [75]:
#training data
train_ones = np.ones(x.shape[0]).reshape(-1, 1)
x = np.append(x, train_ones, axis=1)

#test data
test_ones = np.ones(test_data.shape[0]).reshape(-1, 1)
test_data = np.append(test_data, test_ones, axis=1)

In [76]:
print(train_data.shape)
print(x.shape)
print(y.shape)
print(test_data.shape)

(379, 14)
(379, 105)
(379,)
(127, 105)


In [77]:
from sklearn import model_selection
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y)

In [78]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(284, 105)
(95, 105)
(284,)
(95,)


In [79]:
import pandas as pd
df = pd.DataFrame(x)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,95,96,97,98,99,100,101,102,103,104
0,-0.407850,-0.487722,-1.266023,-0.272599,-0.576134,1.239974,0.840122,-0.520264,-0.752922,-1.278354,...,0.331930,1.537535,0.705805,0.270675,0.566892,1.634190,0.091866,0.168569,1.205582,1.0
1,-0.407374,-0.487722,0.247057,-0.272599,-1.016689,0.001946,-0.838337,0.336351,-0.523001,-0.060801,...,1.033656,0.000004,0.702809,0.113132,0.273531,0.003697,0.012776,0.084779,0.270893,1.0
2,0.125179,-0.487722,1.015999,-0.272599,1.367490,-0.439699,0.687212,-0.577309,1.661245,1.530926,...,1.870030,0.193335,0.472260,0.333285,2.759736,2.343736,0.650565,14.408063,0.794016,1.0
3,0.028304,-0.487722,1.015999,-0.272599,1.859875,-0.047918,0.801005,-0.712836,1.661245,1.530926,...,3.459136,0.002296,0.641610,0.508136,2.759736,2.343736,0.650565,0.004363,0.046414,1.0
4,-0.412408,-0.487722,-0.969827,-0.272599,-0.913029,-0.384137,-0.834781,0.300508,-0.752922,-0.957633,...,0.833622,0.147561,0.696859,0.090305,0.566892,0.917061,0.000423,0.185825,0.000841,1.0


# Gradient Descent for N features

In [80]:
def step_gradient(X_train, Y_train, learning_rate, m,j):
    # Calculate new slope for jth feature
    m_j = 0
    n_data_pts = X_train.shape[0]
    N = len(m)
    for i in range(n_data_pts):
        # calculate the formula m1xi(1)+m2xi(2)+...
        x_i = X_train[i, :]
        y_i = Y_train[i]
        temp_sum = 0
        for k in range(N):
            temp_sum += m[k]*x_i[k]
        ### sub y_i from temp sum
        temp_sum = y_i - temp_sum
        ## complete formula
        m_j += (-2/n_data_pts) * (temp_sum) * x_i[j]
    # update m[j] and return
    m[j] = m[j] - (learning_rate*m_j)
    return m[j]

In [81]:
def gd(x_train, y_train, learning_rate, num_iterations):
    m = [0]* (x_train.shape[1])
    N = len(m)
    #put c=1
    m[-1] = 1
    
    for i in range(num_iterations):
        for j in range(N):
            m[j] = step_gradient(x_train, y_train, learning_rate, m , j)
        print(i, 'Cost : ', cost(x_train, y_train, m))
    return m

In [82]:
def cost(X_train, Y_train, m):
        # This will calculate mean square error
    cost = 0
    n_data_pts = len(X_train)
    N = len(m)
    for i in range(n_data_pts):
        x_i = X_train[i, :]
        y_i = Y_train[i]
        temp_sum = 0
        for k in range(N):
            temp_sum += m[k]*x_i[k]
        temp_sum = y_i - temp_sum
        cost += (1/n_data_pts) * ((temp_sum)**2)
    return cost

In [83]:
def pred(x_test, m):
    reshape_train = x_train.shape[1]
    reshape_m = np.array(m).reshape(reshape_train, 1)
    return np.dot(x_test, reshape_m)

In [84]:
def score(y_truth, y_pred):
    u = ((y_truth - y_pred)**2).sum()
    v = ((y_truth - y_truth.mean())**2).sum()
    return 1 - u/v

In [87]:
def run():
    
    learning_rate = 0.01
    num_iterations = 1000
    m = gd(x, y, learning_rate, num_iterations)
    print('M : ', m)
    
    #to check training score (not evaluation score)
    #y_train_pred = pred(x_train, m)
    #s = score(y, y_train_pred)
    #print(s)
    #to print y_pred
    #y_pred = pred(x_test, m)
    #print(y_pred)
    y_pred = pred(test_data, m)
    print(y_pred)
    np.savetxt('gd_boston_output.csv',y_pred,fmt ='%.5f', delimiter = ',')

In [88]:
run()

0 Cost :  291.0192669088228
1 Cost :  222.42682225085792
2 Cost :  190.69949804227147
3 Cost :  169.13343572481884
4 Cost :  152.15372956578238
5 Cost :  138.14888146234617
6 Cost :  126.38062860072276
7 Cost :  116.38247384336951
8 Cost :  107.81667182268492
9 Cost :  100.42531755819817
10 Cost :  94.00624115664722
11 Cost :  88.39830830096983
12 Cost :  83.47152782932774
13 Cost :  79.12004206279842
14 Cost :  75.25698885051709
15 Cost :  71.81063927469063
16 Cost :  68.72143676113008
17 Cost :  65.93969190664662
18 Cost :  63.42376567072958
19 Cost :  61.13862327248326
20 Cost :  59.05467382502524
21 Cost :  57.14683296464836
22 Cost :  55.39376128513524
23 Cost :  53.77724254494079
24 Cost :  52.28167378830379
25 Cost :  50.89364561507293
26 Cost :  49.60159544506257
27 Cost :  48.39552015582985
28 Cost :  47.266737208905596
29 Cost :  46.20768551789013
30 Cost :  45.21175899621133
31 Cost :  44.273167058145845
32 Cost :  43.38681741232467
33 Cost :  42.5482173414073
34 Cost :  41.

272 Cost :  14.595688186879995
273 Cost :  14.572120219308925
274 Cost :  14.54867020508871
275 Cost :  14.525337189857973
276 Cost :  14.502120231503925
277 Cost :  14.479018399932073
278 Cost :  14.45603077684143
279 Cost :  14.433156455505165
280 Cost :  14.410394540556446
281 Cost :  14.3877441477794
282 Cost :  14.365204403905103
283 Cost :  14.342774446412276
284 Cost :  14.320453423332776
285 Cost :  14.298240493061508
286 Cost :  14.276134824170907
287 Cost :  14.25413559522971
288 Cost :  14.232241994625868
289 Cost :  14.21045322039355
290 Cost :  14.188768480044164
291 Cost :  14.167186990401193
292 Cost :  14.14570797743884
293 Cost :  14.124330676124332
294 Cost :  14.103054330263618
295 Cost :  14.081878192350883
296 Cost :  14.060801523421114
297 Cost :  14.039823592906085
298 Cost :  14.018943678493498
299 Cost :  13.998161065989338
300 Cost :  13.977475049183159
301 Cost :  13.956884929716319
302 Cost :  13.936390016953203
303 Cost :  13.915989627855154
304 Cost :  13.

538 Cost :  10.815719569751495
539 Cost :  10.807227665908893
540 Cost :  10.798761848782581
541 Cost :  10.790322004380029
542 Cost :  10.78190801936182
543 Cost :  10.773519781037109
544 Cost :  10.76515717735887
545 Cost :  10.756820096919277
546 Cost :  10.748508428945257
547 Cost :  10.74022206329375
548 Cost :  10.731960890447375
549 Cost :  10.723724801509904
550 Cost :  10.715513688201904
551 Cost :  10.707327442856219
552 Cost :  10.699165958413834
553 Cost :  10.691029128419396
554 Cost :  10.682916847017022
555 Cost :  10.674829008946098
556 Cost :  10.666765509537012
557 Cost :  10.658726244707024
558 Cost :  10.650711110956202
559 Cost :  10.642720005363248
560 Cost :  10.634752825581515
561 Cost :  10.626809469834896
562 Cost :  10.618889836914045
563 Cost :  10.610993826172113
564 Cost :  10.603121337521149
565 Cost :  10.595272271427998
566 Cost :  10.58744652891057
567 Cost :  10.579644011533954
568 Cost :  10.571864621406679
569 Cost :  10.564108261176953
570 Cost :  

810 Cost :  9.194494950312082
811 Cost :  9.190351831376004
812 Cost :  9.186218142219346
813 Cost :  9.182093852402705
814 Cost :  9.177978931612676
815 Cost :  9.173873349661159
816 Cost :  9.169777076484676
817 Cost :  9.165690082143835
818 Cost :  9.161612336822593
819 Cost :  9.15754381082769
820 Cost :  9.153484474588021
821 Cost :  9.14943429865399
822 Cost :  9.145393253696916
823 Cost :  9.141361310508424
824 Cost :  9.13733843999979
825 Cost :  9.133324613201388
826 Cost :  9.129319801262074
827 Cost :  9.125323975448573
828 Cost :  9.121337107144907
829 Cost :  9.117359167851752
830 Cost :  9.113390129185941
831 Cost :  9.109429962879803
832 Cost :  9.105478640780607
833 Cost :  9.101536134849974
834 Cost :  9.097602417163376
835 Cost :  9.09367745990943
836 Cost :  9.08976123538948
837 Cost :  9.085853716016878
838 Cost :  9.081954874316597
839 Cost :  9.078064682924527
840 Cost :  9.074183114587044
841 Cost :  9.070310142160313
842 Cost :  9.066445738609909
843 Cost :  9.0